# import lib and test GPU of tensorflow


In [16]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
# import matplotlib.image as mping
import matplotlib.animation as animation
import time, datetime
import sys
import gc
import os
import tensorflow as tf
import h5py
from random import randint
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn import manifold, datasets
from IPython.display import clear_output
from tsnecuda import TSNE
#import tsnecuda
#tsnecuda.test()


In [2]:
os.environ["QT_API"] = "pyqt5"
print(os.environ.get('QT_API'))


pyqt5


In [3]:
%matplotlib qt


In [4]:
#test tesnsorflow
print("tensorflow version: ",tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))
# Returns whether TensorFlow was built with CUDA (GPU) support.
print("tf.test.is_Built_With_cuda = ",tf.test.is_built_with_cuda())
# RetUrns whether TensorFlow can access a GPU. (deprecated)
print("tf.test.is_gpu_available = ",tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))


tensorflow version:  2.1.0
Num GPUs Available:  1
Num CPUs Available:  1
tf.test.is_Built_With_cuda =  True
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


tf.test.is_gpu_available =  True


# def function


In [5]:
def show_picture(img, figsizeX=6, figsizeY=4):
    plt.figure(figsize=(figsizeY, figsizeX))
    plt.axis("off")
    # plt.imshow(img, plt.cm.gray, vmax=1.0)
    plt.imshow(img, plt.cm.gray)


# Main program -------------->
# build data path index


In [6]:
def build_data_path_index():
    print("build data path index.....")
    ## read file path
    # folder_path = 'H:/mushroom/VL/Mat/*'                                             # Tommy windows
    # folder_path = '/media/tommy/本機磁碟1/mushroom/VL/Mat/*'                          # Tommy Linux
    folder_path = '/home/shawn/DATA/Special_topic/Pleurotus_eryngii_complete/VL/Mat/*' # Shawn Linux
    folder_path_list = sorted(glob.glob(folder_path))[1:] #excpet 20200507A
    # print('\n'.join(folder_path_list))
    file_name_p = []
    for path in folder_path_list: #folder_name_i = folder_name_index
        file_name_p += sorted(glob.glob(path+'/*'))
        # break

    file_name = [[],[]]
    # category string
    # CS=37 #Tommy Windows
    # CS=53 #Tommy Linux
    CS=83 #Shawn Linux
    for name in file_name_p:
        if name[CS] == 'A': #change to your index
            file_name[0].append(name)
        elif name[CS] == 'B':
            file_name[1].append(name)

    print("build successful")
    return file_name, CS


In [8]:
# file_name : file_name[0] -----> rank of A
#             file_name[1] -----> rank of B
file_name, CS = build_data_path_index()

print()
print("Rank A of number is ", len(file_name[0]))
print("Rank B of number is ", len(file_name[1]))
TFnum = len(file_name[0]) +len(file_name[1]) # Total file number
print("total data number is ", len(file_name[0]) +len(file_name[1]))


build data path index.....
build successful

Rank A of number is  55
Rank B of number is  48
total data number is  103


# del data to picture to observe Placement method


# Process data
OSP --> mask --> get Pleurotus_eryngii --> finish remove background


In [9]:
def select_U_value():
    # select undesired or interfering signatures
    U =               cube_arr[:,305,284]
    U = np.vstack((U, cube_arr[:,680,440]))

    U = np.transpose(U)
    np.save('/home/shawn/DATA/Special_topic/program/Pleurotus_eryngii_complete/U_value.npy',U)
    print("U.shpae = ", U.shape)

def OSP_preprocess(data, select=70):
    ## change data(band, y, x)to
    ##        cube_arr([band][y*x])

    ## $select how band will input to OSP

    cube_arr = data[0]
    cube_arr = cube_arr[np.newaxis,:]
    (band, x, y) = data.shape
    step = band/select
    index = -1
    for i in range(select):
        index = index+step
        # print(index)
        cube_arr = np.concatenate((cube_arr, [data[round(index)]]), axis=0)

    ## if youe need to reselect U_value uncomment it
    # select_U_value(cube_arr)

    (band, x, y) = cube_arr.shape
    cube_arr = cube_arr.reshape(band, x*y)
    return cube_arr


def rho(arr):
    sum=0
    for i in arr:
        sum += i**2
    r = sum**0.5
    return r
def OSPfn(M, U):
    (l, p) = M.shape
    P_U = np.identity(l) - np.dot(U,np.linalg.pinv(U));
    nu = np.zeros((p,1))
    for i in range(p):
        nu[i] = rho(P_U.dot(M[:,i]))
    return nu


In [110]:
def process_data():
    U = np.load('/home/shawn/DATA/Special_topic/program/Pleurotus_eryngii_complete/U_value.npy')
    crop_x = 850
    crop_y = 820
    print("processing....")
    for i in range(2):
        for j in range(len(file_name[i])):
            j=10
            print("i={}/2 j={:2d}/{}\t {}\t".format(i+1,j+1,len(file_name[i]),file_name[i][j][CS-4:]),end="")
            ## read data

            temp = h5py.File(file_name[i][j], 'r').get('cube')
            data = np.array(temp)
            data = data[:, :crop_y, :crop_x]

            ### ========================OSP data ========================
            ## if you change value of select which in OSP_preprocess(), you must to reset U_value.npy manually
            print("OSP\t", end="")
            cube_arr = OSP_preprocess(data)
            osp_out = OSPfn(cube_arr, U).reshape((820,850))
            # show_picture(osp_out)

            ### ======================== get mask ========================
            print("get_mask\t", end="")

            mask=(osp_out/np.max(osp_out))>0.4

            blurred =cv2.medianBlur(np.float32(mask), 5)
            kernel = np.ones((25,25),np.uint8)
            blurred=cv2.morphologyEx(blurred, cv2.MORPH_OPEN, kernel)

            blurred = np.array(blurred,np.uint8)
            blurred=np.clip(blurred, 0, 255)
            contours, _ = cv2.findContours(blurred, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

            ### ======================== build mask ========================
            print("build_mask\t", end="")
            for contours_i in range(len(contours)):
                clone=np.zeros([crop_y,crop_x],dtype=np.int16)
                clone=cv2.drawContours(clone, [contours[contours_i]], contourIdx=-1, color=(255,255,255), thickness=-1)
                clone//=255

                # show_picture(clone)


            print()
            if(j==10):
                break
        break
    print("processing successful")
    return data ,contours


data, contours = process_data()
print(data.shape)


processing....
i=1/2 j=11/55	 0514A10_12.mat	

OSP	

get_mask	build_mask	
processing successful
(300, 820, 850)


# test


In [111]:

crop_x = 850
crop_y = 820
print("build_mask\t", end="")
for contours_i in range(len(contours)):
    clone=np.zeros([crop_y,crop_x],dtype=np.int16)
    clone=cv2.drawContours(clone, [contours[contours_i]], contourIdx=-1, color=(255,255,255), thickness=-1)
    clone//=255
    show_picture(clone)

    (band, x, y) = data.shape
    # cruve_value_list = np.ar

    for band_i in range(band):
        for x_i in range(x):
            for y_i in range(y):
                if(clone[x_i][y_i] == 1):

                    # print(a)
                    pass
    # show_picture(clone)



build_mask	

IndexError: index 820 is out of bounds for axis 0 with size 820

In [102]:
data.shape

(300, 820, 850)

In [101]:
print(type(clone))
print(clone.shape)
print(np.max(clone))
print(np.min(clone))


<class 'numpy.ndarray'>
(800, 850)
1
0
